<a href="https://colab.research.google.com/github/traderjoevitamins/librarian/blob/main/librarian2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_hub
!pip install tensorflow_text
#!pip install tensorflow_decision_forests

##import tensorflow_hub as hub
##import tensorflow_text as text
import tensorflow as tf

import os
import numpy as np
import pandas as pd
import math


In [3]:
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
nyt = pd.read_csv('/content/nyt(3)_embedded_1k_reviews.csv')
gr = pd.read_csv('/content/goodreads_embedded_1k_reviews.csv')


In [4]:
#renaming the openai embeddings
copy = gr['embedding']
gr['openai_embed'] = gr['embedding']
gr = gr[['ISBN13', 'synopsis', 'enjoyed', 'openai_embed']]
nyt['openai_embed'] = nyt['embedding']
nyt = nyt[['titleleft', 'isbn13left', 'synopsis', 'openai_embed']]

In [5]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

class embed:
    def preprocess(self, readerdf, suggestiondf):
      reader_processed = bert_preprocess_model(readerdf['synopsis'])
      sugg_processed = bert_preprocess_model(suggestiondf['synopsis'])
      return reader_processed, sugg_processed

    def encode(self, reader, suggested):
      reader_emb = bert_model(reader)
      sugg_emb = bert_model(suggested)
      return reader_emb, sugg_emb


The shape for gr will be 82 by 128 because we have 82 entries. same goes for the number of entries in nyt at the time of viewing. 128 is the max of sentence. Each array corrosponds to a sentence.

input mask notes
- CLS token in the begining of the sentence SEP token in the end of the sentence

input type ids
- all 0's but this is normal because 0 represents context and 1 represents questions

input word ids
- CLS = 101
- SEP = 102

gr_bert_preprocess.keys() \
nyt_bert_preprocess.keys()



In [6]:
embed_instanance = embed()
reader_processed, suggested_processed = embed_instanance.preprocess(gr, nyt)
reader_embed, sugg_embed = embed_instanance.encode(reader_processed, suggested_processed)


embeddings /
encoder outputs
- len of 12 because model is small
- output of each indvidual encoder
- last encoder output is literally the sequence output

sequence output
- first number = the number of sentences
- second number = 128
- third number = 728 size vector for each word

pooled output
- embedding for entire sentence

default
-

In [7]:
def addtodf(reader, suggested):
  reader['bert_pooled'] = [np.array(i).tolist() for i in reader_embed['pooled_output']]
  #numpy.array(i).tolist()
  suggested['bert_pooled'] = [i.numpy() for i in sugg_embed['pooled_output']]
  reader = pd.DataFrame(reader)
  suggested = pd.DataFrame(suggested)
  return reader, suggested

gr, nyt = addtodf(gr, nyt)

##### **Time to Experiment!**
We are going to see if individual word embeddings (sequence) give different predictions than sentence embeddings (pooled)

independent variables: \
open ai embeddings \
sequence output (bert) \
pooled output (bert) \

In [8]:
#from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from statistics import mean

In [9]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [10]:
def modprep(reader, suggested): ##essentially makes the trainig data
  grx = np.array([i for i in reader['bert_pooled']])
  gry = np.array([])
  if (mean(reader['enjoyed']) == 1 or mean(reader['enjoyed']) == 0):
    gry = np.array(np.random.randint(2, size = (82,1)))
  else:
    gry = np.array([i for i in reader['enjoyed']])

  nytx = np.array([i for i in suggested['bert_pooled']])

  return (grx, gry, nytx)

In [11]:
x_train, y_train, x_test = modprep(gr, nyt)
model.fit(x_train, y_train)

<ipython-input-11-a7497fda7b40>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


RandomForestClassifier(random_state=42)

In [18]:
nyt['predictions'] = model.predict(x_test)
def asklib(suggested):
  full_recs = suggested[suggested['predictions'] == 1]
  titles = full_recs['titleleft']
  return titles



In [19]:
asklib(nyt)

0             WITCH AND WIZARD
4                       FALLEN
11                  CROSS FIRE
12      UNDER THE DOME, PART 1
14           U IS FOR UNDERTOW
                 ...          
1057          THE HUNGER GAMES
1061                    FALLEN
1065          THE TIGER'S WIFE
1070       THE LEGO IDEAS BOOK
1071                NIGHT ROAD
Name: titleleft, Length: 338, dtype: object